In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import uniform, norm

Select your file containing the errors of a model

In [ ]:
def descriptive_statistics():

    mean_a = sum(pa)/n_test
     


    std_a = np.std(pa, ddof=1)


    q1_a = np.percentile(pa, 25)
    med_a = np.percentile(pa, 50)
    q3_a = np.percentile(pa, 75)

In [ ]:
def standard_error(): # erreur standard paramétrique
   
   mean_diff = np.mean(diff)

   se_diff = np.std(diff)/(n_test**0.5)

   return mean_diff, se_diff

def standard_error_boot(): # erreur standard déterminée par bootstrap de la moyenne

   B = 250
   bootstrap_indices = np.random.choice(n_test, size=(B, n_test), replace=True)
   bootstrap = []

   for indices in bootstrap_indices:
      boot_mean = np.mean(diff[indices])
      bootstrap.append(boot_mean)

   mean_diff_boot = np.mean(bootstrap)
   se_boot_diff = np.std(bootstrap)

   return mean_diff_boot, se_boot_diff

def confidence_interval_mean(): # intervalle de confiance par student

    mean_A, se_A= standard_error()

    t_critical = 1.96
    lower_A = mean_A - t_critical * se_A
    upper_A = mean_A + t_critical * se_A

    interval_A = pd.Interval(left=lower_A, right=upper_A, closed='both')

    return interval_A

def confidence_interval_perc_boot(): # intervalle de confiance par percentile bootstrap

   alpha = 0.05
   B = 250
   bootstrap_indices = np.random.choice(n_test, size=(B, n_test), replace=True)
   moy_boot_A = []

   for indices in bootstrap_indices:
      boot_mean = np.mean(pa[indices])
      moy_boot_A.append(boot_mean)

   lower_A = np.percentile(moy_boot_A, 100 * (alpha / 2))
   upper_A = np.percentile(moy_boot_A, 100 * (1 - alpha / 2))

   interval_A_boot = pd.Interval(left=lower_A, right=upper_A, closed='both')


   return interval_A_boot

def confidence_interval_basic_boot(): # intervalle de confiance par basic bootstrap
    
   alpha = 0.05
   B = 250

   bootstrap_indices = np.random.choice(n_test, size=(B, n_test), replace=True)

   moy_boot_A = []

   for indices in bootstrap_indices:
      boot_mean = np.mean(pa[indices])
      moy_boot_A.append(boot_mean)

   lower_A = np.percentile(moy_boot_A, 100 * (alpha / 2))
   upper_A = np.percentile(moy_boot_A, 100 * (1 - alpha / 2))

   interval_A_boot = pd.Interval(left=2*np.mean(pa)-upper_A, right=2*np.mean(pa)-lower_A, closed='both')

   return interval_A_boot

def bca_boot(alpha=0.05): # bootstrap acceléré
    
    B = 250
    bootstrap_indices = np.random.choice(n_test, size=(B, n_test), replace=True)
    sample_means = []

    for indices in bootstrap_indices:
      
      boot_mean = np.mean(pa[indices])
      sample_means.append(boot_mean)
    
    original_mean = np.mean(pa)

    # Acceleration
    jackknife_samples = np.array([np.mean(np.delete(pa, i)) for i in range(len(pa))])
    jackknife_mean = np.mean(jackknife_samples)
    a = np.sum((jackknife_mean - jackknife_samples) ** 3) / (6 * np.sum((jackknife_mean - jackknife_samples) ** 2) ** 1.5)

    # Bias correction
    z0 = stats.norm.ppf(np.mean(sample_means < original_mean))

    z_alpha = norm.ppf(alpha / 2)
    z_1_alpha = norm.ppf(1 - alpha / 2)
    
    lower_percentile = norm.cdf(z0 + (z0 + z_alpha) / (1 - a * (z0 + z_alpha))) * 100
    upper_percentile = norm.cdf(z0 + (z0 + z_1_alpha) / (1 - a * (z0 + z_1_alpha))) * 100

    bca_interval_A = np.percentile(sample_means, [lower_percentile, upper_percentile])

    return bca_interval_A

In [ ]:
def plot_intervals():

    interval_A_boot, interval_B_boot, interval_diff_boot = confidence_interval_basic_boot()
    interval_A_boot_perc, interval_B_boot_perc, interval_diff_boot_perc = confidence_interval_perc_boot()
    interval_A, interval_B, interval_diff = confidence_interval_mean()
    bca_interval_A, bca_interval_B, bca_interval_diff = bca_boot()

    def extract_bounds(intervals):
        if isinstance(intervals, pd.Interval):
            return [intervals.left], [intervals.right]
        elif all(isinstance(interval, pd.Interval) for interval in intervals):
            lower_bounds = [interval.left for interval in intervals]
            upper_bounds = [interval.right for interval in intervals]
            return lower_bounds, upper_bounds
        else:
            raise TypeError("All elements must be pandas Interval objects or a single Interval object")

    y1, y2 = extract_bounds(interval_A)
    y3, y4 = extract_bounds(interval_A_boot)
    y5, y6 = extract_bounds(interval_A_boot_perc)
    y7, y8 = bca_interval_A[0], bca_interval_A[1]

    print("Intervalle de confiance A:", [y1,y2])
    print("Intervalle de confiance A, basic bootstrap:", [y3,y4])
    print("Intervalle de confiance A, percentile bootstrap:", [y5,y6])
    print("Intervalle de confiance A, accelerated bootstrap:", [y7,y8])

    x = np.linspace(0,1,1)

    plt.figure(figsize=(10, 6))
    plt.axhline(y1, label='lower bound mean', marker='o', color="lightblue")
    plt.axhline(y2, label='upper bound mean', marker='o', color="darkblue")
    plt.axhline(y3, label='lower bound basic boot', marker='o', color="brown")
    plt.axhline(y4, label='upper bound basic boot', marker='o', color="darkred")
    plt.axhline(y5, label='lower bound perc boot', marker='o', color="lightgreen")
    plt.axhline(y6, label='upper bound perc boot', marker='o', color="darkgreen")
    plt.axhline(y7, label='lower bound bca boot', marker='o', color="yellow")
    plt.axhline(y8, label='upper bound bca boot', marker='o', color="orange")

    plt.xlabel('')
    plt.ylabel('Interval confidence of the mean A')
    plt.title('Boostrapped confidence intervals of mean A, beta2 = 0.16')
    plt.legend()
    plt.grid(True)
    plt.show()

In [ ]:
def statistical_testing(): # Paired test, testing the null hypothesis mu_D = 0

        mean_diff, se_diff = standard_error()
        mean_diff_boot, se_diff_boot = standard_error_boot()

        test_stat = mean_diff/se_diff

        student_right = stats.t.ppf(0.95, df=n_test-1)
        p_value_right = 1 - stats.t.cdf(test_stat, df=n_test-1)
        puissance_right = 1 - stats.norm.cdf(student_right - test_stat, 0, 1)


        test_stat_boot = mean_diff_boot/se_diff_boot

        p_value_right_boot = 1 - stats.t.cdf(test_stat_boot, df=n_test-1)
        puissance_right_boot = 1 - stats.norm.cdf(student_right - test_stat_boot, 0, 1)

        return p_value_right, p_value_right_boot, puissance_right, puissance_right_boot